# Import

### Librerie

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from keras.layers import *
from sklearn.metrics import *
from keras.optimizers import Adam
from statsmodels.tsa.seasonal import seasonal_decompose
import keras.backend as K
import keras
from keras import layers
import time

### Dataset

In [ ]:
dataset = pd.read_csv('../dati_continui_con_id.csv')

In [ ]:
dataset[160:180]

In [ ]:
# carico elenco utenti per scorrere tutti gli id
elenco_utenti = pd.read_csv('../../dataset/userinfo.csv')
elenco_utenti.drop(elenco_utenti.columns[[0, 3]], axis=1, inplace=True) # drop colonne superflue
elenco_utenti.columns = ['user_id', 'timezone', 'sex', 'age', 'height']
elenco_utenti.head()

In [ ]:
dataset.set_index(['user_id','date'], inplace=True)
dataset

In [ ]:
# creo una lista di di dataframe dove ogni elemento è l'insieme delle date per utente
utenti = []

# inserisco per ogni elemento della lista il dataframe relativo ad un utente presente nel dataset
for i, r in elenco_utenti.iterrows():
    utente = r['user_id']
    if utente in dataset.index:
        utenti.append(dataset.loc[utente])

In [ ]:
# elimina stagionalità
senza_stag = []
for corrente in utenti:
    if len(corrente) >=14:
        lista_stag = []
        lista_stag.append(corrente[['steps']].squeeze() - seasonal_decompose(corrente[['steps']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['bedin']].squeeze() - seasonal_decompose(corrente[['bedin']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['bedout']].squeeze() - seasonal_decompose(corrente[['bedout']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['sleep duration']].squeeze() - seasonal_decompose(corrente[['sleep duration']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['deepduration']].squeeze()  - seasonal_decompose(corrente[['sleep duration']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['lightduration']].squeeze() - seasonal_decompose(corrente[['sleep duration']], model='additive', period = 7).seasonal)
        frame = {'steps' : lista_stag[0], 'bedin' : lista_stag[1], 'bedout' : lista_stag[2], 'sleep duration' : lista_stag[3], 'deepduration' : lista_stag[4], 'lightduration' : lista_stag[5]}
        senza_stag.append(pd.DataFrame(frame))
utenti = senza_stag

In [ ]:
# Funzione normalizzazione
scaler = MinMaxScaler(feature_range=(0,1))
def normalizza(dat):
    df_for_training_scaled = scaler.fit_transform(dat)
    return df_for_training_scaled

In [ ]:
# Adesso normalizza ogni dataset
utenti_normalizzati = []
for u in utenti:
    utenti_normalizzati.append(normalizza(u))

In [ ]:
#n_past is the number of step we will look in the past to predict the next target value.
col2pred = 5  # 0:steps, 1:bedin, 2:bedout, 3:sleep duration, 4:deep duration, 5:lightduration
window_size = 7
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            #per ogni feature fa l'append dei precedenti
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])  
            dataY.append(dataset[i,col2pred])
    return np.array(dataX),np.array(dataY)

In [ ]:
# crea insieme di train
train_lista = utenti_normalizzati[:7000]

trainX_lista = []
trainY_lista = []

for t in train_lista:
    trainX_temp, trainY_temp = createXY(t,window_size)
    if len(trainX_temp.shape) == 3 and len(trainY_temp.shape) == 1:
        trainX_lista.append(trainX_temp)
        trainY_lista.append(trainY_temp)

trainX = np.concatenate(trainX_lista)
trainY = np.concatenate(trainY_lista)

In [ ]:
# crea insieme di test
test_lista = utenti_normalizzati[7000:]

testX_lista = []
testY_lista = []

for t in test_lista:
    testX_temp, testY_temp = createXY(t,window_size)
    if len(testX_temp.shape) == 3 and len(testY_temp.shape) == 1:
        testX_lista.append(testX_temp)
        testY_lista.append(testY_temp)

testX = np.concatenate(testX_lista)
testY = np.concatenate(testY_lista)

In [ ]:
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

In [ ]:
# features num
features_num = dataset.shape[1]
features_num

# Metriche di valutazione (MAE-MSE-RMSE)

In [ ]:
def metrics(true_value_train, pred_value_train, true_value_test, pred_value_test):
    print("**********TRAIN-SET***********")
    print("MAE: {:.3f}".format(mean_absolute_error(
        true_value_train, pred_value_train)))
    print("MSE: {:.3f}".format(
        mean_squared_error(true_value_train, pred_value_train)))
    print("RMSE: {:.3f}".format(
        mean_squared_error(true_value_train, pred_value_train, squared=True)))
    # metrics test set
    print("**********TEST-SET***********")
    print("MAE: {:.3f}".format(mean_absolute_error(
        true_value_test, pred_value_test)))
    print("MSE: {:.3f}".format(
        mean_squared_error(true_value_test, pred_value_test)))
    print("RMSE: {:.3f}".format(
        mean_squared_error(true_value_test, pred_value_test, squared=True)))
    

    plt.plot(true_value_test[:150], color='red', label='Real light duration')
    plt.plot(pred_value_test[:150], color='blue', label='Predicted light duration')
    plt.title('Light duration Prediction')
    plt.xlabel('Time')
    plt.ylabel('Light duration')
    plt.legend()
    plt.show()

# Data rescaling 

In [ ]:
def values4metrics(model, trainX, trainY, testX, testY):
    prediction_train = model.predict(trainX)
    prediction_train_copies_array = np.repeat(prediction_train, features_num, axis=-1)
    pred_train = scaler.inverse_transform(np.reshape(prediction_train_copies_array, (len(prediction_train), features_num)))[:, col2pred]
    true_value_train_copies_array = np.repeat(trainY, features_num, axis=-1)
    true_value_train = scaler.inverse_transform(np.reshape(true_value_train_copies_array, (len(trainY), features_num)))[:, col2pred]

    prediction_test = model.predict(testX)
    prediction_copies_test_array = np.repeat(prediction_test,features_num, axis=-1)
    pred_test = scaler.inverse_transform(np.reshape(prediction_copies_test_array,(len(prediction_test),features_num)))[:,col2pred]
    true_value_test_copies_array = np.repeat(testY,features_num, axis=-1)
    true_value_test = scaler.inverse_transform(np.reshape(true_value_test_copies_array,(len(testY),features_num)))[:,col2pred]
    
    return true_value_train, pred_train, true_value_test, pred_test

# Architetture di supporto

In [ ]:
class Time2Vec(Layer):

    def __init__(self, output_dim=None, **kwargs):
        self.output_dim = output_dim
        super(Time2Vec, self).__init__(**kwargs)

    def build(self, input_shape):

        self.W = self.add_weight(name='W',
                                shape=(input_shape[-1], self.output_dim),
                                initializer='uniform',
                                trainable=True)

        self.P = self.add_weight(name='P',
                                shape=(input_shape[1], self.output_dim),
                                initializer='uniform',
                                trainable=True)

        self.w = self.add_weight(name='w',
                                shape=(input_shape[1], 1),
                                initializer='uniform',
                                trainable=True)

        self.p = self.add_weight(name='p',
                                shape=(input_shape[1], 1),
                                initializer='uniform',
                                trainable=True)

        super(Time2Vec, self).build(input_shape)

    def call(self, x):

        original = self.w * x + self.p
        sin_trans = K.sin(K.dot(x, self.W) + self.P)

        return K.concatenate([sin_trans, original], -1)

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(input_shape,head_size,num_heads,ff_dim,num_transformer_blocks,mlp_units,dropout=0,mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="tanh")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

input_shape = trainX.shape[1:]

model_transf = build_model(input_shape, head_size=256, num_heads=4, ff_dim=4, num_transformer_blocks=4, 
                        mlp_units=[128], mlp_dropout=0.4, dropout=0.25)

model_transf.compile(loss="mse", optimizer = Adam(learning_rate=1e-4))
model_transf.summary()

# Funzione per richiamare i modelli

In [ ]:
def model_recurrent(type = 'gru'):
    model = Sequential()
    
    if type == 'gru': # GRU
        model.add(GRU(256, return_sequences=True, input_shape=(window_size, features_num)))
        model.add(GRU(128, return_sequences=True))
        model.add(GRU(64))
        model.add(Dropout(0.25))
        model.add(Dense(32, activation = 'tanh'))
        model.add(Dropout(0.5))
        
    if type == 'lstm': # LSTM
        model.add(LSTM(256, return_sequences=True,input_shape=(window_size, features_num)))
        model.add(LSTM(128, return_sequences=True))
        model.add(LSTM(64))
        model.add(Dropout(0.25))
        model.add(Dense(32, activation='tanh'))
        model.add(Dropout(0.5))
    
    if type == 'bidirectional':  # Bidirectional-LSTM
        model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(window_size,features_num)))
        model.add(Bidirectional(LSTM(128, return_sequences=True)))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dropout(0.25))
        model.add(Dense(32, activation='tanh'))
        model.add(Dropout(0.5))   

    if type == 'time2vec':  # Time2Vec-LSTM
        model.add(Input(shape=(window_size, features_num)))
        model.add(Time2Vec(120))
        model.add(LSTM(256, return_sequences=True))
        model.add(LSTM(128, return_sequences=True))
        model.add(LSTM(64))
        model.add(Dropout(0.25))
        model.add(Dense(32, activation='tanh'))
        model.add(Dropout(0.5))  
        
    if type == 'cnn':  # CNN-LSTM
        model.add(Conv1D(filters=256, kernel_size=2, activation='tanh', input_shape=(window_size,features_num)))
        model.add(Conv1D(filters=128, kernel_size=2, activation='tanh'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(RepeatVector(30))
        model.add(LSTM(units=100, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(units=100, return_sequences=True))
        model.add(Dense(100, activation='tanh'))        

    model.add(Dense(1))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model 

# Iperparametri di allenamento

In [ ]:
n_batch = 4096  # 8192
n_epochs = 100

# GRU

In [ ]:
model_gru = model_recurrent('gru')
model_gru.summary()

In [ ]:
start = time.time()
history_gru = model_gru.fit(trainX, trainY, epochs=n_epochs, batch_size=n_batch, verbose=1)
end = time.time()
time_gru = end-start;

In [ ]:
true_value_train_gru, pred_value_train_gru, true_value_test_gru, pred_value_test_gru = values4metrics(model_gru, trainX, trainY, testX, testY)

# LSTM

In [ ]:
model_lstm = model_recurrent('lstm')
model_lstm.summary()

In [ ]:
start = time.time()
history_lstm = model_lstm.fit(trainX, trainY, epochs = n_epochs, batch_size = n_batch, verbose=1)
end = time.time()
time_lstm = end-start;

In [ ]:
true_value_train_lstm, pred_value_train_lstm, true_value_test_lstm, pred_value_test_lstm = values4metrics(model_lstm, trainX, trainY, testX, testY)

# Bidirectional-LSTM

In [ ]:
model_bidirec = model_recurrent('bidirectional')
model_bidirec.summary()

In [ ]:
start = time.time()
history_bidirect = model_bidirec.fit(trainX, trainY, epochs = n_epochs, batch_size = n_batch, verbose = 1)
end = time.time()
time_bi = end-start;

In [ ]:
true_value_train_bi, pred_value_train_bi, true_value_test_bi, pred_value_test_bi = values4metrics(model_bidirec, trainX, trainY, testX, testY)

# Time2Vec-LSTM

In [ ]:
model_t2v = model_recurrent('time2vec')
model_t2v.summary()

In [ ]:
start = time.time()
history_t2v = model_t2v.fit(trainX, trainY, epochs = n_epochs, batch_size = n_batch, verbose = 1)
end = time.time()
time_t2v = end-start;

In [ ]:
true_value_train_t2v, pred_value_train_t2v, true_value_test_t2v, pred_value_test_t2v = values4metrics(model_t2v, trainX, trainY, testX, testY)

# CNN-LSTM

In [ ]:
model_cnn = model_recurrent('time2vec')
model_cnn.summary()

In [ ]:
start = time.time()
history_cnn = model_cnn.fit(trainX, trainY, epochs = n_epochs,  batch_size = n_batch, verbose=1)
end = time.time()
time_cnn = end-start;

In [ ]:
true_value_train_cnn, pred_value_train_cnn, true_value_test_cnn, pred_value_test_cnn = values4metrics(model_cnn, trainX, trainY, testX, testY)

# T2V-Transformer 

In [ ]:
model_transf.summary()

In [ ]:
start = time.time()
history_t = model_transf.fit(trainX, trainY, epochs = n_epochs, batch_size = n_batch, verbose = 1)
end = time.time()
time_t = end-start;

In [ ]:
true_value_train_transf, pred_value_train_transf, true_value_test_trasnf, pred_value_test_transf = values4metrics(model_transf, trainX, trainY, testX, testY)

# Plot risultati 

In [ ]:
print("--- GRU ---\n\nTime: " + str(time_gru))
metrics(true_value_train_gru, pred_value_train_gru, true_value_test_gru, pred_value_test_gru)

In [ ]:
print("--- LSTM ---\n\nTime: " + str(time_lstm))
metrics(true_value_train_lstm, pred_value_train_lstm, true_value_test_lstm, pred_value_test_lstm)

In [ ]:
print("--- Bidirectional ---\n\nTime: " + str(time_bi))
metrics(true_value_train_bi, pred_value_train_bi, true_value_test_bi, pred_value_test_bi)

In [ ]:
print("--- Time2Vec - LSTM ---\n\nTime: " + str(time_t2v))
metrics(true_value_train_t2v, pred_value_train_t2v, true_value_test_t2v, pred_value_test_t2v)

In [ ]:
print("--- CNN - LSTM ---\n\nTime: " + str(time_cnn))
metrics(true_value_train_cnn, pred_value_train_cnn, true_value_test_cnn, pred_value_test_cnn)

In [ ]:
print("--- Transformer ---\n\nTime: " + str(time_t))
metrics(true_value_train_transf, pred_value_train_transf, true_value_test_transf, pred_value_test_transf)